# Kaggle House Prices Challenge

## House Prices: Advanced Regression Techniques

Predict sales prices and practice feature engineering, RFs, and gradient boosting

(Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview)

First, we start by importing the libraries needed for this project.

In [0]:
import pandas as pd
import numpy as np

Loading the data into dataframes.

In [0]:
test = pd.read_csv("test.csv")
train = pd.read_csv("train.csv")

In [19]:
test.shape, train.shape

((1459, 80), (1460, 81))

In [0]:
test.columns, train.columns

(Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
        'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
        'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
        'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
        'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
        'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
        'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
        'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
        'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
        'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
        'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
        'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
        'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
        'GarageCond

We can see that the test dataset contains one more variable compared to the train dataset - which is the "SalePrice" variable. In our analysis / prediction this serves as the dependent variable we want to predict given the houses' characteristics.

In [0]:
train.SalePrice.describe()

count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

## Data preprocessing and cleaning

We'll start the Data Cleaning by checking if the dependent variable in the test dataset contains any missing values.

In [0]:
train.SalePrice.isnull().sum()

0

All observations contain data for the target variable, therefore we can continue by taking a look at all the other variables contained in the train and test dataset.

In [0]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])

miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])

In [0]:
missings_train.head(20)

,Total,Percent
PoolQC,1453,0.995205
MiscFeature,1406,0.963014
Alley,1369,0.937671
Fence,1179,0.807534
FireplaceQu,690,0.472603
LotFrontage,259,0.177397
GarageCond,81,0.055479
GarageType,81,0.055479
GarageYrBlt,81,0.055479
GarageFinish,81,0.055479


In [0]:
missings_test.head(20)

,Total,Percent
PoolQC,1456,0.997944
MiscFeature,1408,0.965045
Alley,1352,0.926662
Fence,1169,0.801234
FireplaceQu,730,0.500343
LotFrontage,227,0.155586
GarageCond,78,0.053461
GarageQual,78,0.053461
GarageYrBlt,78,0.053461
GarageFinish,78,0.053461


As a rule of thumb we completely ignore columns that contain at least 15% missing values and will not try to impute the missing values with any kind of computation, e.g. using means. Therefore, we will delete the variables "PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu" and "LotFrontage".

In [0]:
train = train.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])

The variables "GarageCond", "GarageType", "GarageQual", "GarageYrBlt" and "GarageFinish" contain exactly the same number of missing values, which seems kind of odd. Therefore, we'll take a closer look at these variables.

In [0]:
for var in ["GarageCond", "GarageType", "GarageQual", "GarageYrBlt", "GarageFinish"]:
    print(pd.crosstab(index=train[var], columns="count"))

We can see that for "GarageCond" and "GarageQual" the most frequently occurring value is "TA", which means that the condition and quality of the garages are average/typical. We will replace the missing values of these two variables therefore with "TA" as well. The variable "GarageYrBlt" refers to the year in which the garage was built. Since we also have the year in which the houses themselves are built we can drop this variable without losing much explaining information. In addition to that we also drop the "GarageFinish" and "GarageType" variable.

In [0]:
train = train.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
train["GarageCond"] = train.GarageCond.fillna(value="TA")
train["GarageQual"] = train.GarageQual.fillna(value="TA")

In the same way as above we take a closer look at the "Bsmt*" variables.

In [0]:
for var in ["BsmtFinType2", "BsmtExposure", "BsmtCond", "BsmtFinType1", "BsmtQual"]:
    print(pd.crosstab(index=train[var], columns="count"))

col_0         count
BsmtFinType2       
ALQ              19
BLQ              33
GLQ              14
LwQ              46
Rec              54
Unf            1256
col_0         count
BsmtExposure       
Av              221
Gd              134
Mn              114
No              953
col_0     count
BsmtCond       
Fa           45
Gd           65
Po            2
TA         1311
col_0         count
BsmtFinType1       
ALQ             220
BLQ             148
GLQ             418
LwQ              74
Rec             133
Unf             430
col_0     count
BsmtQual       
Ex          121
Fa           35
Gd          618
TA          649


We delete the "BsmtFinType*" variables since these are highly subjective and do not add much information to our model. The missing values of "BsmtCond" will be imputed with the most common value "TA". The rows containing missing values for "BsmtQual" and "BsmtExposure" will be deleted from the dataset.

In [0]:
train = train.drop(columns=["BsmtFinType1", "BsmtFinType2"])
train.BsmtCond = train["BsmtCond"].fillna(value="TA")
for var in ["BsmtQual", "BsmtExposure"]:
    train = train.drop(train.loc[train[var].isnull()].index)

The variable "Electrical" contains only 1 missing value, therefore we only delete this specific row of data. We proceed in the same way with "MasVnrType" and "MasVnrArea".

In [0]:
for var in ["Electrical", "MasVnrType", "MasVnrArea"]:
    train = train.drop(train.loc[train[var].isnull()].index)

Running the above code again to check if all missing values are deleted.

In [0]:
miss_count_train = train.isnull().sum().sort_values(ascending=False)
perc_miss_train = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missings_train = pd.concat([miss_count_train, perc_miss_train], axis=1, keys=["Total", "Percent"])
missings_train

,Total,Percent
SalePrice,0,0.0
ExterQual,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
MasVnrArea,0,0.0
ExterCond,0,0.0
YearBuilt,0,0.0


This looks good.

We now handled all the missing data on the training set. As the next step we will clean the test data. Since we want to evaluate our model on Kaggle after finishing the modeling, we cannot drop any observations because we need predicted house prices for each row of the test data. Therefore, we will impute the missing data in the test dataset with the most frequent category for categorical features and the mean for numerical features.

In [0]:
test = test.drop(columns=["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu", "LotFrontage"])
test = test.drop(columns=["GarageYrBlt", "GarageFinish", "GarageType"])
test["GarageCond"] = test.GarageCond.fillna(value="TA")
test["GarageQual"] = test.GarageQual.fillna(value="TA")
test = test.drop(columns=["BsmtFinType1", "BsmtFinType2"])
test.BsmtCond = test["BsmtCond"].fillna(value="TA")


In [25]:
test.shape, train.shape

((1459, 69), (1413, 70))

In [0]:
# categorical:
for var in ["BsmtExposure", "BsmtQual", "MasVnrType", "MSZoning", "Utilities", "Functional", "SaleType", "Exterior2nd", "Exterior1st", "KitchenQual"]:
    test[var] = test[var].fillna(value=test[var].value_counts().index[0])

# numerical
for var in ["MasVnrArea", "BsmtHalfBath", "BsmtFullBath", "BsmtUnfSF", "GarageArea", "GarageCars", "BsmtFinSF1", "BsmtFinSF2", "TotalBsmtSF"]:
    test[var] = test[var].fillna(test[var].mean())

In [0]:
miss_count_test = test.isnull().sum().sort_values(ascending=False)
perc_miss_test = (test.isnull().sum() / test.isnull().count()).sort_values(ascending=False)
missings_test = pd.concat([miss_count_test, perc_miss_test], axis=1, keys=["Total", "Percent"])
missings_test

,Total,Percent
SaleCondition,0,0.0
MasVnrArea,0,0.0
YearRemodAdd,0,0.0
RoofStyle,0,0.0
RoofMatl,0,0.0
Exterior1st,0,0.0
Exterior2nd,0,0.0
MasVnrType,0,0.0
ExterQual,0,0.0
BsmtUnfSF,0,0.0


In [27]:
test.shape, train.shape

((1459, 69), (1413, 70))

Perfect. We do not have any missing values in our test set anymore and have kept all observations, for which we gonna predict the House Sale Price.

For now we will not create any additional features and just use the features that are already in our dataset. The only feature we are going to create is a variable containing the total area of the living space.

In [0]:
train["TotalSF"] = train["TotalBsmtSF"] + train["GrLivArea"]
test["TotalSF"] = test["TotalBsmtSF"] + test["GrLivArea"]

We will also take a look at the skewness and the kurtosis of the numerical feature to see if they are (at least somehow close to) normally distributed.

In [0]:
num_cols = train.dtypes != "object"
skewness_train = (train.loc[:, num_cols].skew()).sort_values(ascending=False) 
skewness_test = (test.loc[:, num_cols].skew()).sort_values(ascending=False)

In [0]:
skewness_train, skewness_test

(PoolArea         14.127094
 3SsnPorch         7.780393
 LowQualFinSF      7.455332
 MiscVal           5.247312
 KitchenAbvGr      4.345562
 BsmtHalfBath      3.857352
 ScreenPorch       3.084680
 BsmtFinSF2        2.464986
 EnclosedPorch     2.099524
 HalfBath          0.534201
 MasVnrArea        0.466378
 BsmtFullBath      0.381061
 2ndFlrSF          0.274074
 MSSubClass        0.256100
 Fireplaces        0.155582
 SalePrice         0.142198
 1stFlrSF          0.132100
 WoodDeckSF        0.119664
 YrSold            0.104427
 GrLivArea         0.038953
 TotalSF           0.037185
 TotRmsAbvGrd     -0.005868
 OpenPorchSF      -0.050993
 LotArea          -0.147690
 OverallCond      -0.232525
 TotalBsmtSF      -0.269386
 FullBath         -0.394370
 OverallQual      -0.461236
 YearRemodAdd     -0.538216
 YearBuilt        -0.661761
 BsmtFinSF1       -0.699306
 MoSold           -0.910759
 BedroomAbvGr     -1.287130
 GarageCars       -1.350907
 Id               -1.792487
 BsmtUnfSF        -2

We see that there are some variables with a very high skewness in the training data as well as in the test data. The skewness of a distribution is always compared to the skewness of the normal distribution, which is zero. To get our data to look more normal, we will log(feature + 1) transform it.

In [0]:
# log1p transforming the training data
for var in num_cols[num_cols].index:
    train[var] = np.log1p(train[var])

In [0]:
num_cols_test = num_cols.drop(labels="SalePrice")
# log1p transfroming the test data
for var in num_cols_test[num_cols_test].index:
    test[var] = np.log1p(test[var])

In [0]:
train.shape, test.shape

## Model selection 

We continue by building a Machine Learning Pipeline using Scikit Learn. A pipeline object sequentially applies a list of transformers and a final estimator. 


We will play around with different algorithms, tune their hyperparameters using Cross Validation and pick the best performing one.

Before we can work with our data, we first need to create separate dataframes containing our feature variables and the target variable. This needs to be done only for our training data since it is our aim to predict SalePrice for the test data, which is why it is not contained in this data.

In [0]:
y = train.SalePrice.values
X = train.drop("SalePrice", axis=1)

To make predictions and to fit models, the last step that has to be done is to convert all categorical features into numeric ones. This way scikit-learn can handle them. We do this by using pandas get_dummies() function. To make sure we end up with the same number of columns in both the training and the test dataset we first concatenate both, then apply get_dummies() and then separating them again.

In [34]:
X.shape, test.shape

((1413, 70), (1459, 70))

In [0]:
# creating a dummy to distinguish between train and test data
X["train"] = 1
test["train"] = 0
# concatenating dataframes and creating dummies from categorical features
combined = pd.concat([X, test])
df = pd.get_dummies(combined, drop_first=True)



In [36]:
df.shape

(2872, 215)

In [37]:
X = df[df["train"]==1]
X = X.drop("train", axis=1)
test = df[df["train"]==0]
test = test.drop("train", axis=1)
X.shape, test.shape

((1413, 214), (1459, 214))

Before training and tuning our models we separate the data into training and test data to evaluate our models.

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [39]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1130, 214), (283, 214), (1130,), (283,))

 ### k-nearest Neighbors (KNN)

We will start with a relatively simple algorithm - k-nearest Neighbor or KNN.

In [0]:
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

Looking at the data, we notice that are our feature variables' ranges vary substantially between each other. Therefore we will add a transformer to our pipeline which standardizes the data. Standardization centers each variable around zero with unit variance. This is done by subtracting the means from each feature and dividing by its standard deviation.

After that we instantiate our KNN estimator, create a list containing the steps applied by the pipeline and then defining the pipeline.

In [0]:
# instantiate the scaling transformator 
scaler = preprocessing.StandardScaler()
# instantiate the KNN estimator
knn = KNeighborsRegressor(n_neighbors=10)
# creating a list containing the steps the pipeline is to apply
steps_knn =  [("scaler", scaler), ("knn", knn)]
# define the pipeline object
pipeline_knn = Pipeline(steps_knn)

The KNN algorithm has one parameter that can and should be tuned, which is the number of neighbors that should be considered. We will therefore define a dictionary containing all hyperparameters that should be tuned and define the different values that should be tested.

In [0]:
neighbors = {"knn__n_neighbors":list(range(1,21))}

Next we are gonna set up our Cross Validation (CV) object using 5-fold CV and fit it to our data.

In [0]:
cv_knn = GridSearchCV(pipeline_knn, neighbors, cv=5)

In [0]:
cv_knn.fit(X_train, y_train)

In [0]:
cv_knn.best_params_

{'knn__n_neighbors': 12}

In [0]:
print(cv_knn.refit)

From the above output we conclude that the default of the argument "refit" is True. This means that, by default, our CV pipeline automatically refits the model on the entire training set using the best parameters found by CV, which is 12 in our case. Therefore we can now directly use the CV object to make predictions for unseen data.

As in the Kaggle challenge we will use the log of the Root mean squared error metric to evaluate our model's performance on the test set. To do this we first need to predict the sales price for the unseen test data.

In [0]:
from sklearn.metrics import mean_squared_error
y_pred = cv_knn.predict(X_test)
rmse_knn = np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred)))
print("The KNN algorithm with " + str(cv_knn.best_params_) + " yields a (log) RMSE of: " + str(rmse_knn))

The KNN algorithm with {'knn__n_neighbors': 12} yields a (log) RMSE of: 0.197561198068067


### Random forest

After trying out the KNN algorithm, we now continue with the Random Forest Algorithm. Using similar steps as before we will build up a pipeline object applying the transformations and the estimation on our dataset sequentially automatically.

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [0]:
# instantiate the RandomForest Regressor
rf_reg = RandomForestRegressor(random_state=123)
# creating a list containing the steps the pipeline should apply
steps_rf = [("scaler", scaler), ("rf_reg", rf_reg)]
# create the pipeline object
pipeline_rf = Pipeline(steps_rf)

For RandomForests there are a large number of Hyperparameters that can be tuned. In this project we are going to tune the number of trees in the random forest [n_estimators], the number of features considered at every split [max_features], the maximum number of levels in a tree [max_depth], the mininum number of samples required to split a node [min_samples_split], the minimum number of observations required at each leaf node [min_samples_leaf] and if bootstrap should be used training each tree [bootstrap]. 

We first define the ranges for each hyperparameter that should be considered in our CV tuning and then create our search grid containing the before created lists.

In [0]:
n_estimators = list(np.arange(200, 2001, 200))
max_features = ["auto", "sqrt", "log2"]
max_depth = list(np.arange(10, 101, 10))
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4, 8]
bootstrap = [True, False]
param_dist = {"rf_reg__n_estimators": n_estimators,
              "rf_reg__max_features": max_features,
              "rf_reg__max_depth": max_depth,
              "rf_reg__min_samples_split": min_samples_split,
              "rf_reg__min_samples_leaf": min_samples_leaf,
              "rf_reg__bootstrap": bootstrap}

Testing all possible combinations of hyperparameters would ammount to testing $10*3*11*3*3*2 = 5,940$ combinations, instead of testing all of these we will use RandomizedSearchCV to select randomly from our defined distributions which combinations are tested. 

In [0]:
from sklearn.model_selection import RandomizedSearchCV
cv_random_rf = RandomizedSearchCV(pipeline_rf, param_dist, cv=3, n_iter=100)

Just as before with the KNN algorithm we now can fit the pipeline to our data.

In [48]:
cv_random_rf.fit(X_train, y_train)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=Pipeline(memory=None,
                                      steps=[('scaler',
                                              StandardScaler(copy=True,
                                                             with_mean=True,
                                                             with_std=True)),
                                             ('rf_reg',
                                              RandomForestRegressor(bootstrap=True,
                                                                    criterion='mse',
                                                                    max_depth=None,
                                                                    max_features='auto',
                                                                    max_leaf_nodes=None,
                                                                    min_impurity_decrease=0.0,
                          

In [49]:
cv_random_rf.best_params_

{'rf_reg__bootstrap': False,
 'rf_reg__max_depth': 60,
 'rf_reg__max_features': 'sqrt',
 'rf_reg__min_samples_leaf': 2,
 'rf_reg__min_samples_split': 2,
 'rf_reg__n_estimators': 200}

Based on the chosen parameters from RandomizedSearchCV we can now manually decrease the range of the hyperparameters to be tested and use GridSearchCV as before to find the best parameters for our model.

In [50]:
n_estimators_2 = list(np.arange(100, 301, 50))
max_depth_2 = list(np.arange(40, 81, 10))
max_features_2 = ["sqrt"]
min_samples_split_2 = [2, 3]
min_samples_leaf_2 = [1, 2, 3]
bootstrap_2 = [True, False]
param_grid = {"rf_reg__n_estimators": n_estimators_2,
              "rf_reg__max_features": max_features_2,
              "rf_reg__max_depth": max_depth_2,
              "rf_reg__min_samples_split": min_samples_split_2,
              "rf_reg__min_samples_leaf": min_samples_leaf_2,
              "rf_reg__bootstrap": bootstrap_2}
param_grid

{'rf_reg__bootstrap': [True, False],
 'rf_reg__max_depth': [40, 50, 60, 70, 80],
 'rf_reg__max_features': ['sqrt'],
 'rf_reg__min_samples_leaf': [1, 2, 3],
 'rf_reg__min_samples_split': [2, 3],
 'rf_reg__n_estimators': [100, 150, 200, 250, 300]}

We can now instantiate and then fit our GridSearchCV object as before.

In [51]:
cv_rf = GridSearchCV(pipeline_rf, param_grid, cv=3)
cv_rf.fit(X_train, y_train)

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rf_reg',
                                        RandomForestRegressor(bootstrap=True,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                 

In [54]:
cv_rf.best_params_

{'rf_reg__bootstrap': False,
 'rf_reg__max_depth': 40,
 'rf_reg__max_features': 'sqrt',
 'rf_reg__min_samples_leaf': 1,
 'rf_reg__min_samples_split': 2,
 'rf_reg__n_estimators': 200}

In [55]:
print(cv_rf.refit)

True


We see that the parameters chosen by RandomizedSearchCV were already pretty good and did not really need that much of a finetuning. Furthermore, the GridSearchCV function also already refit the RandomForest model on the whole training data using the parameters that are found to work best.

As before, we now can create predictions for our hold out test set and evaluate our model using log RMSE as before.

In [56]:
from sklearn.metrics import mean_squared_error
y_pred_rf = cv_rf.predict(X_test)
rmse_rf = np.sqrt(mean_squared_error(np.log(y_test), np.log(y_pred_rf)))
print("The RandomForest algorithm with " + str(cv_rf.best_params_) + " yields a (log) RMSE of: " + str(rmse_rf))

The RandomForest algorithm with {'rf_reg__bootstrap': False, 'rf_reg__max_depth': 40, 'rf_reg__max_features': 'sqrt', 'rf_reg__min_samples_leaf': 1, 'rf_reg__min_samples_split': 2, 'rf_reg__n_estimators': 200} yields a (log) RMSE of: 0.011345300033064953


Great! We see a clear improvement in our evaluation metric, i.e. the RandomForest Regressor performs much better than the KNN fitted at first.

### Making predictions and creating our submission file

Since we want to check our model's performance in comparison with other people's models, we need to make predictions for the test dataset provided by Kaggle and upload our results to Kaggle. According to the challenge's description, our submission should be a file containing the observation's ID and the predicted SalesPrice. We first create a dataframe and then save it as a .csv file.

In [57]:
ident = list(test.Id)
print(ident)

[7.2875606403097235, 7.288244401020124, 7.288927694521257, 7.289610521451167, 7.290292882446597, 7.2909747781429814, 7.291656209174461, 7.292337176173877, 7.293017679772782, 7.293697720601438, 7.294377299288821, 7.29505641646263, 7.295735072749282, 7.29641326877392, 7.297091005160418, 7.29776828253138, 7.298445101508147, 7.2991214627108, 7.299797366758161, 7.300472814267799, 7.301147805856032, 7.301822342137932, 7.302496423727326, 7.3031700512368, 7.303843225277705, 7.304515946460155, 7.305188215393037, 7.305860032684009, 7.306531398939505, 7.307202314764738, 7.307872780763706, 7.30854279753919, 7.309212365692763, 7.3098814858247865, 7.310550158534422, 7.311218384419629, 7.311886164077165, 7.312553498102598, 7.313220387090301, 7.313886831633462, 7.31455283232408, 7.315218389752975, 7.315883504509785, 7.316548177182976, 7.317212408359839, 7.317876198626496, 7.318539548567902, 7.319202458767849, 7.31986492980897, 7.32052696227274, 7.321188556739479, 7.321849713788356, 7.322510433997394, 

In [0]:
test_pred = list(cv_rf.predict(test))

In [59]:
len(ident), len(test_pred)

(1459, 1459)

In [60]:
submiss = {"Id": ident, "SalePrice": test_pred}
print(submiss)

{'Id': [7.2875606403097235, 7.288244401020124, 7.288927694521257, 7.289610521451167, 7.290292882446597, 7.2909747781429814, 7.291656209174461, 7.292337176173877, 7.293017679772782, 7.293697720601438, 7.294377299288821, 7.29505641646263, 7.295735072749282, 7.29641326877392, 7.297091005160418, 7.29776828253138, 7.298445101508147, 7.2991214627108, 7.299797366758161, 7.300472814267799, 7.301147805856032, 7.301822342137932, 7.302496423727326, 7.3031700512368, 7.303843225277705, 7.304515946460155, 7.305188215393037, 7.305860032684009, 7.306531398939505, 7.307202314764738, 7.307872780763706, 7.30854279753919, 7.309212365692763, 7.3098814858247865, 7.310550158534422, 7.311218384419629, 7.311886164077165, 7.312553498102598, 7.313220387090301, 7.313886831633462, 7.31455283232408, 7.315218389752975, 7.315883504509785, 7.316548177182976, 7.317212408359839, 7.317876198626496, 7.318539548567902, 7.319202458767849, 7.31986492980897, 7.32052696227274, 7.321188556739479, 7.321849713788356, 7.3225104339

In [61]:
submission = pd.DataFrame(submiss)
print(submission.head())
"\n"
"\n"
"\n"
print(submission.tail())

         Id  SalePrice
0  7.287561  11.705338
1  7.288244  11.970325
2  7.288928  12.096652
3  7.289611  12.141390
4  7.290293  12.169190
            Id  SalePrice
1454  7.977968  11.375875
1455  7.978311  11.370731
1456  7.978654  11.990708
1457  7.978996  11.627189
1458  7.979339  12.314755


In [0]:
from google.colab import files

submission.to_csv('submission_rf2.csv', index=False)
# files.download('submission_rf2.csv')

We can now upload our submission file to Kaggle and check how well our model performs on the unseen test data.